In [ ]:
import sys
from models import ASTModel
import dataloader
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import json
import random
from collections import defaultdict
import argparse
from traintest import train, validate

sys.path.append('./src') 
os.environ['TORCH_HOME'] = '../pretrained_models'  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def prepare_ast_dataset(root_path, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2):
    # ตรวจสอบว่าสัดส่วนรวมกันได้ 1.0 หรือไม่
    assert abs((train_ratio + val_ratio + test_ratio) - 1.0) < 1e-5, "Ratios must sum to 1.0"

    # 1. นิยามโครงสร้างข้อมูลและ Class Map
    patient_groups = {
        "Cough_PTB": defaultdict(list),
        "Cough_Non-PTB": defaultdict(list)
    }
    class_map = {"Cough_PTB": "1", "Cough_Non-PTB": "0"}

    # 2. รวบรวมไฟล์เสียงและจัดกลุ่มตาม Patient ID
    for folder_name, label_idx in class_map.items():
        folder_path = os.path.join(root_path, folder_name)
        if not os.path.exists(folder_path):
            print(f"⚠️ Warning: Folder {folder_path} not found.")
            continue
            
        for file in os.listdir(folder_path):
            if file.endswith(".wav"):
                patient_id = file.split('_')[0]
                full_path = os.path.abspath(os.path.join(folder_path, file))
                patient_groups[folder_name][patient_id].append(full_path)

    # 3. รวบรวมรายชื่อผู้ป่วยทั้งหมดและสุ่มแบ่ง 3 กลุ่ม
    all_patients = set()
    for folder_name in patient_groups:
        all_patients.update(patient_groups[folder_name].keys())
    
    all_patients = list(all_patients)
    random.seed(42) # กำหนด seed เพื่อให้ผลการสุ่มคงเดิม
    random.shuffle(all_patients)

    total_patients = len(all_patients)
    train_split_idx = int(total_patients * train_ratio)
    val_split_idx = train_split_idx + int(total_patients * val_ratio)

    train_ids = all_patients[:train_split_idx]
    val_ids = all_patients[train_split_idx:val_split_idx]
    test_ids = all_patients[val_split_idx:]

    # 4. จัดเตรียมข้อมูลลง List สำหรับ JSON และแยก ID ตามคลาสเพื่อการแสดงผล
    train_list, val_list, test_list = [], [], []
    
    # ตัวแปรสำหรับเก็บ ID แยกคลาสเพื่อ Print Summary
    summary_ids = {
        "train_ptb": set(), "train_non_ptb": set(),
        "val_ptb": set(), "val_non_ptb": set(),
        "test_ptb": set(), "test_non_ptb": set()
    }

    for folder_name, groups in patient_groups.items():
        is_ptb = (folder_name == "Cough_PTB")
        
        for p_id, paths in groups.items():
            for path in paths:
                item = {"wav": path, "labels": class_map[folder_name]}
                
                if p_id in train_ids:
                    train_list.append(item)
                    if is_ptb: summary_ids["train_ptb"].add(p_id)
                    else: summary_ids["train_non_ptb"].add(p_id)
                elif p_id in val_ids:
                    val_list.append(item)
                    if is_ptb: summary_ids["val_ptb"].add(p_id)
                    else: summary_ids["val_non_ptb"].add(p_id)
                elif p_id in test_ids:
                    test_list.append(item)
                    if is_ptb: summary_ids["test_ptb"].add(p_id)
                    else: summary_ids["test_non_ptb"].add(p_id)

    # 5. บันทึกไฟล์ JSON (หัวใจสำคัญสำหรับ dataloader.py ของ AST)
    with open('train_data.json', 'w') as f:
        json.dump({"data": train_list}, f, indent=4)
    with open('val_data.json', 'w') as f:
        json.dump({"data": val_list}, f, indent=4)
    with open('test_data.json', 'w') as f:
        json.dump({"data": test_list}, f, indent=4)
    def count_files(data_list):
        ptb_count = sum(1 for item in data_list if item['labels'] == "1")
        non_ptb_count = sum(1 for item in data_list if item['labels'] == "0")
        return ptb_count, non_ptb_count

    train_ptb_audio, train_non_ptb_audio = count_files(train_list)
    val_ptb_audio, val_non_ptb_audio = count_files(val_list)
    test_ptb_audio, test_non_ptb_audio = count_files(test_list)
    # 6. แสดงผลสรุปอย่างละเอียด
    print(" AST DATASET PREPARATION SUMMARY (3-Way Patient-Level Split)")
    print("=" * 70)
    
    print(f"🔹 TRAIN SET (Total Patients: {len(train_ids)})")
    print(f"    PTB ID: {sorted(list(summary_ids['train_ptb']))}")
    print(f"    Non-PTB ID: {sorted(list(summary_ids['train_non_ptb']))}")
    print(f"    Total Audio Files: {len(train_list)} (PTB: {train_ptb_audio} files | Non-PTB: {train_non_ptb_audio} files)")  
    
    print(f"🔸 VALIDATION SET (Total Patients: {len(val_ids)})")
    print(f"    PTB ID: {sorted(list(summary_ids['val_ptb']))}")
    print(f"    Non-PTB ID: {sorted(list(summary_ids['val_non_ptb']))}")
    print(f"    Total Audio Files: {len(val_list)} (PTB: {val_ptb_audio} files | Non-PTB: {val_non_ptb_audio} files)")   
    
    print(f"🔴 TEST SET (Total Patients: {len(test_ids)})")
    print(f"    PTB ID: {sorted(list(summary_ids['test_ptb']))}")
    print(f"    Non-PTB ID: {sorted(list(summary_ids['test_non_ptb']))}")
    print(f"    Total Audio Files: {len(test_list)} (PTB: {test_ptb_audio} files | Non-PTB: {test_non_ptb_audio} files)")
    print("=" * 70)

# เรียกใช้งาน: แบ่ง Train 70%, Validate 10%, Test 20%
prepare_ast_dataset("./Data", train_ratio=0.6, val_ratio=0.2, test_ratio=0.2)

 AST DATASET PREPARATION SUMMARY (3-Way Patient-Level Split)
🔹 TRAIN SET (Total Patients: 9)
    PTB ID: ['006', '008', '012', '015']
    Non-PTB ID: ['001', '002', '003', '013', '014']
    Total Audio Files: 392 (PTB: 151 files | Non-PTB: 241 files)
🔸 VALIDATION SET (Total Patients: 3)
    PTB ID: ['009', '011']
    Non-PTB ID: ['005']
    Total Audio Files: 37 (PTB: 5 files | Non-PTB: 32 files)
🔴 TEST SET (Total Patients: 3)
    PTB ID: ['016']
    Non-PTB ID: ['004', '007']
    Total Audio Files: 83 (PTB: 31 files | Non-PTB: 52 files)


In [5]:
print("🚀 STARTING AST-P SINGLE RUN (80/20 SPLIT) WITH VISUALIZATION")
# 1. กำหนดโฟลเดอร์เก็บผลลัพธ์
exp_dir = './exp/tb_ast_p_single_run'
os.makedirs(f'{exp_dir}/models', exist_ok=True)
MEAN_NORM = -3.2406702
STD_NORM = 4.9710765
# 2. ตั้งค่า Data Config
train_audio_conf = {'num_mel_bins': 128, 'target_length': 100, 'freqm': 24, 'timem': 20, 'mixup': 0.0, 'dataset': 'audioset', 'mode': 'train', 'mean': MEAN_NORM, 'std': STD_NORM, 'noise': False, 'skip_norm': False}
eval_audio_conf = {'num_mel_bins': 128, 'target_length': 100, 'freqm': 0, 'timem': 0, 'mixup': 0.0, 'dataset': 'audioset', 'mode': 'evaluation', 'mean': MEAN_NORM, 'std': STD_NORM, 'noise': False, 'skip_norm': False}
BATCH_SIZE = 8
# 3. สร้าง DataLoader
train_loader = torch.utils.data.DataLoader(
    dataloader.AudiosetDataset('train_data.json', label_csv='class_labels_indices.csv', audio_conf=train_audio_conf),
    batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True
)
eval_loader = torch.utils.data.DataLoader(
    dataloader.AudiosetDataset('eval_data.json', label_csv='class_labels_indices.csv', audio_conf=eval_audio_conf),
    batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=0, pin_memory=True
)
# แสดงภาพตัวอย่าง Spectrogram
# ดึงข้อมูลมา 1 Batch จาก train_loader เพื่อดูภาพ
sample_inputs, sample_labels = next(iter(train_loader))
# sample_inputs จะมีขนาด [batch_size, temporal_frame_num, frequency_bin_num]
# เราดึง index 0 ของ Batch มาพล็อต
spec_data = sample_inputs[0].numpy()  
label_data = torch.argmax(sample_labels[0]).item()
class_name = "PTB" if label_data == 1 else "Non-PTB"

plt.figure(figsize=(10, 4))
# พล็อตโดยสลับแกน (Transpose) ให้ความถี่ (128) อยู่แกน Y และเวลา (100) อยู่แกน X
plt.imshow(spec_data.T, aspect='auto', origin='lower', cmap='viridis')
plt.title(f"Example Log Mel Filterbank Spectrogram (Class: {class_name})")
plt.ylabel("Frequency Bins (128)")
plt.xlabel("Time Frames (100)")
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.show()
print("☝️ ภาพด้านบนคือฟีเจอร์ Spectrogram 128 มิติที่ผ่านการทำ Normalization แล้ว ซึ่งจะถูกส่งเข้า AST Model")
# ==========================================

# 4. สร้าง AST-P Model 
ast_model = ASTModel(
    label_dim=2, 
    fstride=10, 
    tstride=10, 
    input_fdim=128, 
    input_tdim=100, 
    imagenet_pretrain=True, 
    audioset_pretrain=True, 
    model_size='base384'
)

# 5. ตั้งค่า Training Arguments
args = argparse.Namespace(
    exp_dir=exp_dir, dataset='audioset', n_class=2, lr=1e-5, n_epochs=30, batch_size=BATCH_SIZE, 
    n_print_steps=10, save_model=True, loss='CE', metrics='mAP', 
    lrscheduler_start=10, lrscheduler_step=5, lrscheduler_decay=0.5, 
    warmup=True, wa=True, wa_start=1, wa_end=30
)

# 6. เริ่มกระบวนการ Train
print("\nStarting Model Training...")
train(ast_model, train_loader, eval_loader, args)

# 7. โหลด Best Model เพื่อประเมินผล
best_model_path = f'{args.exp_dir}/models/best_audio_model.pth'
best_model = ASTModel(label_dim=2, fstride=10, tstride=10, input_fdim=128, input_tdim=100, imagenet_pretrain=False, audioset_pretrain=False, model_size='base384', verbose=False)
best_model = torch.nn.DataParallel(best_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = best_model.to(device)

best_model.load_state_dict(torch.load(best_model_path, map_location=device))

stats, eval_loss = validate(best_model, eval_loader, args, epoch='best')


🚀 STARTING AST-P SINGLE RUN (80/20 SPLIT) WITH VISUALIZATION


NameError: name 'torch' is not defined

In [ ]:
# ==========================================
# 📊 [NEW] วาด Confusion Matrix จากชุด Eval
# ==========================================
from sklearn.metrics import confusion_matrix


print("\n📈 Generating Confusion Matrix...")
all_preds = []
all_targets = []

best_model.eval() # เข้าสู่โหมดทดสอบ
with torch.no_grad():
    for audio, labels in eval_loader:
        audio = audio.to(device)
        # AST Output เป็น Raw Logits (ไม่มี Sigmoid/Softmax)
        logits = best_model(audio) 
        # ใช้ Argmax เพื่อหาคลาสที่มีค่า Logit สูงที่สุด
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        
        all_preds.extend(preds)
        
        # ✅ แก้ไขบรรทัดนี้: แปลง labels ให้เป็นตัวเลขเดี่ยว (0 หรือ 1) ก่อนเก็บลงลิสต์
        all_targets.extend(torch.argmax(labels, dim=1).numpy())

# สร้างกราฟ Confusion Matrix
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(6, 5))
sys.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-PTB (0)', 'PTB (1)'], 
            yticklabels=['Non-PTB (0)', 'PTB (1)'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('AST-P Confusion Matrix (Eval Set)')
plt.tight_layout()
plt.show()

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    roc_curve, auc, average_precision_score, confusion_matrix
)

# ==========================================
# 1. หา Best Epoch จากไฟล์ result.csv ของ AST
# ==========================================
# โครงสร้างคอลัมน์มาตรฐานของ AST: [mAP, mAUC, precision, recall, d_prime, train_loss, valid_loss, cum_mAP, cum_mAUC, lr]
csv_path = './exp/tb_ast_p_single_run/result.csv'
df_results = pd.read_csv(csv_path, header=None)

# คอลัมน์ index 1 คือ mAUC ของแต่ละ Epoch
best_epoch = df_results[1].idxmax() + 1
best_mauc = df_results[1].max()
print(f"🌟 Best Epoch: {best_epoch} (Validation AUC from log: {best_mauc:.4f})\n")

# ==========================================
# 2. เก็บผลลัพธ์แบบความน่าจะเป็น (Probabilities) จาก Best Model
# ==========================================
print(" Evaluating Best Model on Evaluation Set...")
all_probs = []
all_targets = []

best_model.eval() # เข้าสู่โหมดทดสอบ
with torch.no_grad():
    for audio, labels in eval_loader:
        audio = audio.to(device)
        logits = best_model(audio) 
        
        # AST Output เป็น Raw Logits (มี 2 ค่า ตามจำนวน n_class=2)
        # เราใช้ Softmax เพื่อแปลงเป็นความน่าจะเป็น (0-1) และเลือกเฉพาะความน่าจะเป็นของคลาส 1 (PTB)
        probs = torch.softmax(logits, dim=-1)[:, 1].cpu().numpy()
        
        # ดึง Label คลาสจริง
        targets = torch.argmax(labels, dim=1).cpu().numpy()
        
        all_probs.extend(probs)
        all_targets.extend(targets)

all_probs = np.array(all_probs)
all_targets = np.array(all_targets)

# ==========================================
# 3. คำนวณความแม่นยำ AUROC, AUPRC และหา Best Threshold
# ==========================================
fpr, tpr, roc_thresholds = roc_curve(all_targets, all_probs)
auroc_val = auc(fpr, tpr)
auprc_val = average_precision_score(all_targets, all_probs)


# ==========================================
# 4. คำนวณ Sensitivity และ Specificity ที่จุด Best Threshold
# ==========================================
# สร้างคำทำนายใหม่โดยอิงจากจุด Threshold ที่ดีที่สุด แทนที่จะเป็น 0.5
optimal_preds = (all_probs >= 0.5).astype(int)
tn, fp, fn, tp = confusion_matrix(all_targets, optimal_preds).ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(" Performance Metrics:")
print(f" - AUROC:             {auroc_val:.4f}")
print(f" - AUPRC:             {auprc_val:.4f}")
print(f" - Sensitivity (TPR): {sensitivity:.4f}  (TP:{tp}, FN:{fn})")
print(f" - Specificity (TNR): {specificity:.4f}  (TN:{tn}, FP:{fp})")

# 5. วาดกราฟ ROC Curve และ Confusion Matrix ให้อยู่คู่กัน
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ---- กราฟที่ 1: ROC Curve ----
axes[0].plot(fpr, tpr, color='darkorange', lw=2.5, label=f'AUROC = {auroc_val:.4f}')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# มาร์คจุด Threshold ที่ดีที่สุด
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
axes[0].set_ylabel('True Positive Rate (Sensitivity)', fontsize=12)
axes[0].set_title('Receiver Operating Characteristic (ROC) Curve', fontsize=14)
axes[0].legend(loc="lower right", fontsize=11)
axes[0].grid(True, linestyle='--', alpha=0.6)

# ---- กราฟที่ 2: Confusion Matrix ----
cm_optimal = confusion_matrix(all_targets, optimal_preds)
sns.heatmap(cm_optimal, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-PTB (0)', 'PTB (1)'], 
            yticklabels=['Non-PTB (0)', 'PTB (1)'],
            annot_kws={"size": 14}, ax=axes[1])
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_title(f'Confusion Matrix (@ Threshold {0.5:.4f})', fontsize=14)

plt.tight_layout()
plt.show()